In [ ]:
# Checar GPU disponível
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Importar o algoritmo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Compilar usando GPU disponível
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [ ]:
# Configurar a rede para o treinamento 
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [ ]:
# Criar um diretório para os pesos ajustados
!mkdir "/mydrive/yolov3"

In [ ]:
# Definir as classes
!echo -e 'Anacardium occidentale\nBauhinia variegata\nClitoria fairchildiana' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Importar pesos
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
# Extrair as imagens tratadas
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
import glob
img_lista = glob.glob("data/obj/*.jpg")
print(img_lista)

In [ ]:
# Gerar o arquivo training.txt
file = open("data/train.txt", "w") 
file.write("\n".join(img_lista)) 
file.close() 

In [ ]:
# Iniciar treinamento
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show -map